# DABN13 - Assignment 7

## Part 1: Canned procedures for p-value adjustment
In this part we are going to explore which variables are relevant for first run U.S. box office ($) sales, for a set of 62 movies.
We have 12 explanatory variables:

* MPRating = MPAA Rating code, 1=G, 2=PG, 3=PG13, 4=R,
* Budget = Production budget ($Mil),
* Starpowr = Index of star poser,
* Sequel = 1 if movie is a sequel, 0 if not,
* Action = 1 if action film, 0 if not,
* Comedy = 1 if comedy film, 0 if not,
* Animated = 1 if animated film, 0 if not,
* Horror = 1 if horror film, 0 if not,
* Addict = Trailer views at traileraddict.com,
* Cmngsoon = Message board comments at comingsoon.net,
* Fandango = Attention at fandango.com (see Example 4.12),
* Cntwait3 = Percentage of Fandango votes that can't wait to see



In [21]:
import pandas as pd
import statsmodels.api as sm
import numpy as np
import os

#os.chdir("change working directory if needed")

movie_data = pd.read_csv("movie_buzz.txt")

### Task 1a)

We want to train a linear regression model with the *logarithm* of box office sales as output and all other variables in `movie_data` (plus a constant) as inputs. We are not going to hold out any validation data. 

First, extract the output and input variables into $n \times 1$ and $n \times p$ NumPy arrays `y_1a` and `X_1a`, respectively. Second, use the `OLS()` function in `statsmodels` to specify the regression model and to learn its coefficients. Save the specified model as `lm_fit_1a`. Third, save the p-values of tests for the individual significance of the regression coefficients as `pvalues_1a`. 

*Hint:* Information about where the p-values of a learned linear regression can be found is provided by the [documentation](https://www.statsmodels.org/dev/generated/statsmodels.regression.linear_model.RegressionResults.html#statsmodels.regression.linear_model.RegressionResults) of the `RegressionResults()` class in `statsmodels`.

In [22]:
# CODE_CHUNK code_chunk_01
# 1.
y_1a = np.log(movie_data["BOX"].values).reshape(-1, 1)
X_1a = movie_data.drop(columns=["BOX"]).values
X_1a = sm.add_constant(X_1a)

# 2.
lm_fit_1a = sm.OLS(y_1a, X_1a).fit()
print(lm_fit_1a.summary())

# 3.
pvalues_1a = lm_fit_1a.pvalues


                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.616
Model:                            OLS   Adj. R-squared:                  0.522
Method:                 Least Squares   F-statistic:                     6.554
Date:                Wed, 29 Oct 2025   Prob (F-statistic):           8.52e-07
Time:                        21:21:02   Log-Likelihood:                -54.149
No. Observations:                  62   AIC:                             134.3
Df Residuals:                      49   BIC:                             162.0
Df Model:                          12                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         15.2537      0.598     25.512      0.0

### Task 1b)

The `multitest()` function in the `stats.multipletest` module of `statsmodels` calculates p-value adjustments that control either the familywise error rate (FWER) or the false discovery rate (FDR). A list of the supported methods is given in the [statsmodels documentation](https://www.statsmodels.org/dev/generated/statsmodels.stats.multitest.multipletests.html).

Use this function to correct the p-values `p_val_1a` using Holm's procedure and save them as p_holm_1b. Then, store the names of all input variables whose corrected p-value is below the significance level `alpha_1b` in an object `names_selected_1b`.


In [23]:
# CODE_CHUNK code_chunk_02
import statsmodels.stats.multitest as smm

alpha_1b = 0.05

# 1.
p_holm_1b = smm.multipletests(pvalues_1a, alpha=alpha_1b, method='holm')[1]

# 2.
names = np.array(['const'] + list(movie_data.drop(columns=['BOX']).columns))
names_selected_1b = names[p_holm_1b < alpha_1b]
print(names_selected_1b)

['const' 'CNTWAIT3 ']


## Part 2: The FWER in a simulation study

In this part, we will use simulations to convince ourselves that standard p-values are problematic if we conduct a larger number of tests and that simple p-value adjustments can address our multiple testing problem.

Below, I have written a short function `X_fun_precompute` that calculates particular functions of some provided matrix `XX`. These are returned as a dictionary. We will need this function output in the tasks below.


In [24]:
def X_fun_precompute(XX):
    p1        = XX.shape[1]
    XtX       = XX.T @ XX
    XtX_inv   = np.linalg.solve(XtX, np.eye(p1))
    XtX_inv_Xt = XtX_inv @ XX.T  
    results = {'X': XX,
               'XtX': XtX,
               'XtX_inv': XtX_inv,
               'XtX_inv_Xt': XtX_inv_Xt}
    return(results)

### Task 2a)
We start with the individual building blocks of our simulation study. First, we obtained learned coefficients, residuals and variance estimates from a linear regression model. Do the following:

1. Call the function `X_fun_precompute` with `X_1a` as its only argument and save the function output as `X_funs_2a`.
2. Manually compute the learned coefficients in a regression model with `y_1a` as output and `X_1a` as inputs. Save it as `beta_hat_2a`. Use the pre-computed functions of `X_1a` inside `X_funs_2a` instead of constructing them again.  
3. Obtain the marginal variance 
$$ \widehat{Var}[\hat{\beta} | X, y]= \hat{\sigma}^{2}(X^TX)^{-1} $$
where $\hat{\sigma}^2$ is the estimated variance of the residuals. Save it as `VX_2a`. Again, use objects inside `X_funs_2a` to refer to `X_1a` and any function that involves `X_1a` alone. 

*Hint:* A good way to see that you have done things correctly is to compare your result with the summary of your regression results in Task 1a).

In [ ]:
# CODE_CHUNK code_chunk_03
# 1.
X_funs_2a = ??

# 2.
??

# 3.
??

### Task 2b)

As a next step, do the following:

1. Obtain $t-$ statistics for $\beta_i$ under $H_0:\beta_i=0$, namely
$$
t_i = \left|\frac{\hat{\beta}_i}{\widehat{sd[\hat{\beta}_i | X, y]}} \right|,
$$
and save them as `t_2b`. Use existing objects from Task 2a to arrive there.
2. Compute the corresponding $p-$values of two-sided tests and save them as `pvalues_2b`. Here, keep in mind that $\frac{\hat{\beta}_i}{\widehat{sd[\hat{\beta}_i | X, y]}} $ follows a Student-t distribution with $n-p$ degrees of freedom. The cumulative density function of this distribution is given by the `t.cdf()` function in the stats-module of `scipy`. See its documentation [here](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.t.html)

*Hint*: Use the result
$$
P_{0}(|T| \geq t) = P_{0}(T \leq -t)  + P_{0}(T \geq t).
$$


In [ ]:
# CODE_CHUNK code_chunk_04
from scipy.stats import t

# 1.
t_2b       = ??

# 2.
pvalues_2b = ??

### Task 2c)
Now write a function `calculate_p_2c`.  Its arguments are a $n times 1$ Numpy array of output values `y0` as well as a dictionary `X0_funs`  with the same structure as `X_funs_2a`. `calculate_p_2c` should conduct the same steps as in Tasks 2a-b and should return the p-values of a regression of `y0` on the input matrix from which `X0_funs` has been generated.

In [ ]:
# CODE_CHUNK code_chunk_05
??

### Task 2d)
Now we can set op our simulation study. Write a loop that repeats the following steps 1000 times:

1. Simulate an output `y_fake` unrelated to the existing inputs `X_1a`. `y_fake` should be drawn from a normal distribution with mean and variance coming from the sample mean and variance of the observed $y$ in your dataset.

2. Use your function `calculate_p_2c` to compute $p-$values in a linear regression of `y_fake` on the observed inputs `X_1a`. Use the existing dictionary `X_funs_2a` as second function argument.
3. Adjust the p-values returned by `calculate_p_2c` in two different ways: i) Bonferroni adjustment, ii) Holm's method.
4. Store the number of variables in the learned model (excluding the intercept) whose coefficient is found to be statistically significant at a significance level $\alpha=0.05$. Do this for i) p-value without adjustment, ii) Bonferroni-adjusted p-values, iii) Holm-adjusted p-values.

For step 3., create empty arrays, `n_selected_naive`, `n_selected_Holm` and `n_selected_Bonferroni` before you write the for loop. Inside the loop, store the number of statistically significant inputs in your *j*-th iteration in element *j* of these objects.

When the loop has finished, calculate the FWER using results from the 100 iterations of your loop. Save the FWER for your three sets of p-value based selection of significant predictors as `FWER_Naive`, `FWER_Holm` and `FWER_BF`.

Does either of the three methods successfully control the FWER? Save your answer in the string variable `FWER_control_2d`.

In [ ]:
# CODE_CHUNK code_chunk_06
??

FWER_control_2d = "??"

## Part 3: Test power with p-value adjustment

Now that we have created an efficient algorithm for computing the p-values, we are going to explore what happens when one adds several signals to the simulated data.
However, we first rescale all input variables in our data so that they have a variance of one.

In [7]:
from sklearn.preprocessing import StandardScaler

scaler_3a = StandardScaler(with_mean=False)
X_3a      = scaler_3a.fit_transform(X_1a)

### Task 3a)
Next, we simulate artificial outcomes. However, in contrast to task 2d we are *not* creating new $y$ that are completely unrelated to the observed predictors in our box office sales data. More specifically, we simulate $y$ from the linear model
$$
\mathbf{y}= \mathbf{X}\boldsymbol{\beta} + \boldsymbol{\varepsilon}
$$
where $X$ contains the scaled predictors of Task 3a. For the vector of slope coefficients $\beta$, we let $\beta_{2:5}=log(2:5)$ whereas all other elements of this vector are 0. The model errors $\epsilon$ are drawn independently from a standard normal distribution. Construct a new object `y_sim_3a` that contains simulated outcomes from the model described here.

In [ ]:
# CODE_CHUNK code_chunk_07
signal_index_3a = [1, 2, 3, 4]
??

### Task 3b)
In this task were are getting adjusted p-values again:

1. Obtain a list object `x_funs_3b` containing functions of `X_3a` using `X_fun_precompute()`.
2. Use your`calculate_p_2c()` function to get p-values for significance tests in a regression with `y_sim_3a` as output and `X_3a` as inputs. Save your result as `pvalues_3b`.
2. Get adjusted p-values from Bonferroni and Benjamini & Hochberg corrections by using `p.adjust()`. Save these p-values as `p_BF_3b` and `p_hochberg_3b`.

In [ ]:
# CODE_CHUNK code_chunk_08
# 1.
x_funs_3b  = ??

# 2.
pvalues_3b = ??

# 3.
p_BF_3b       = ??
p_hochberg_3b = ??

### Task 3c)
Next, we calculate the false discovery proportion of Bonferroni and Benjamini-Hochberg methods.

1. Create  binary vectors `selected_BF_3c` and `selected_hochberg_3c` whose elements indicate which input variables are significant at a (familywise) significance level of $\alpha=0.15$. 
2. Create  binary vectors `selected_true_BF_3c` and `selected_true_hochberg_3c` whose elements indicate if the coefficient on a particular input variable is significant *and* has a nonzero true value in the setup that you used to generate `y_sim_3a`. 
3. Use the objects from steps 1 and 2 to calculate the $fdp$ for Bonferroni and Benjamini & Hochberg corrections. Save them as `fdp_BF_3c` and `fdp_hochberg_3c` 

In [ ]:
# CODE_CHUNK code_chunk_09
alpha_3c = 0.15

# 1.
selected_BF_3c       = ??
selected_hochberg_3c = ??

# 2.
selected_true_BF_3c       = ??
selected_true_hochberg_3c = ??

# 3.
fdp_BF_3c       = ??
fdp_hochberg_3c = ??

### Task 3d)

We are now going to conduct simulations in order to investigate the $FWER$, $FDR$ and the power of each signal variable in the model setup of Task 3b. We do this in a double loop. 

The inner loop generates 1000 vectors of simulated outputs and records (using a vector of indicator variables) which of the input variables in `X_3a` are significant at a 15% significance level. The underlying p-values are to be corrected using the Bonferroni and Benjamini-Hochberg corrections. 

The outer loop runs through 20 cases with signal variables of increasing signal strength. Signal strength is controlled by multiplying the coefficient vector `beta_sim_3a` of your model  with a magnitude factor `mag` whose 20 possible values are saved in a vector `magnitudes`. After the inner loop has been run, the outer loop uses information of variable selection in every simulated dataset to calculate FDR and FWER, excluding the pvalue of a test for statistical significance of the intercept. The outer loop also stores the power of significance test for coefficients on each of the four signal variables in the model (i.e. those with nonzero beta coefficient).

*Note*: The code chunk below prepares a number of empty vectors and matrices that are to be filled in the inner and outer loops. The matrices starting with `selected.` might be a bit confusing. Keep in mind that they have dimension $\mathrm{sim} \times p$. So each iteration of the inner loop is supposed to fill one of their rows.

In [ ]:
# CODE_CHUNK code_chunk_10
# Don't change anything here:
np.random.seed(4456)

sim = 1000
p = X_3a.shape[1]
magnitudes = np.linspace(0, 1, 20)

Power_Hochberg_3d    = np.zeros((len(magnitudes), len(signal_index_3a)))
Power_BF_3d          = np.zeros((len(magnitudes), len(signal_index_3a)))
FWER_Hochberg_3d     = np.zeros(len(magnitudes))
FWER_BF_3d           = np.zeros(len(magnitudes))
FDR_BF               = np.zeros(len(magnitudes))
FDR_Hochberg         = np.zeros(len(magnitudes))
selected_BF_3d       = np.zeros((sim, p))
selected_Hochberg_3d = np.zeros((sim, p))

# Start changing stuff below
for i, mag in enumerate(magnitudes):
    for ii in range(sim):
        beta_loop_3d          = ??
        y_loop_3d             = ??
        pvals_loop_3d         = ??
        p_BF_loop_3d          = ??
        p_hoch_loop_3d        = ??
        selected_BF_3d[ii, :] = ??
        selected_Hochberg_3d[ii, :] = ??
    
    Power_BF_3d[i, :]       = ??
    Power_Hochberg_3d[i, :] = ??
    FWER_BF_3d[i]           = ??
    FWER_Hochberg_3d[i]     = ??
    FDR_BF[i]               = ??
    FDR_Hochberg[i]         = ??

### Task 3e)
For the variable with the strongest signal (i.e. the largest coefficient in `beta_sim_3a`) create a line plot that plots the power of a significance test on its coefficient against the signal strength `mag`. Since you have power for both Bonferroni amd Benjamini-Hochberg adjustments, you need to include the corresponding two power curves in the same plot.

Then, create another plot in which you repeat the same task for the weakest signal (i.e. the variable with smallest nonzero coefficient in `beta_sim_3a`).

Why is the difference in power between Bonferroni and Benjamini-Hochberg adjusted p-values larger on the weakest signal? Write your answer into the string variable `why_difference_3e`.

In [ ]:
# CODE_CHUNK code_chunk_11
import matplotlib.pyplot as plt

??

why_difference_3e = "??"

#### Task 3f)

Now plot the FWER rates for both Bonferroni and Benamini-Hochberg adjustments against the signal strength `mag`. 
Which pattern can you see in these lines? Write your answer into the string variable `what_pattern_3f`. 
What causes these patterns? Write your answer into the string variable `why_pattern_3f`. 

In [ ]:
# CODE_CHUNK code_chunk_12
??

what_pattern_3f = "??"
why_pattern_3f  = "??"

## Part 4: Knock-off inference

In Mullainathan, Sendhil, and Jann Spiess. 2017. "Machine Learning: An Applied Econometric Approach." *Journal of Economic Perspectives*, 31 (2): 87-106, the authors compared several ML techniques for prediction of house prices (log of house prices). Here we will explore this data sets for variable selection (slightly cleaned by us a priori).
First we load the data.

In [ ]:
#Delete this before uploading
from sklearn.preprocessing import StandardScaler
import pandas as pd
import statsmodels.api as sm
import numpy as np
import os

In [3]:
ahs_data = pd.read_csv("asst7_data_part4.csv")
y_4a = ahs_data['y_4a'].to_numpy()
X_4a = ahs_data.iloc[:,1:]
X_varnames_4a = X_4a.columns

### Task 4a)
As usual when working with lasso, we rescale the variances of all predictors and demean all variables before training any model. Do that here.

In [4]:
# CODE_CHUNK code_chunk_13
scaler_4a = StandardScaler()
X_4a      = scaler_4a.fit_transform(X_4a)
y_4a      = y_4a - y_4a.mean()

### Task 4b)
Now use *scikit-learn* to perform ten-fold cross-validation and save the resulting object as `lasso_cv_4b`.

I have already written some code that saves the tuning parameter value selected through the one-S.E. rule as `alpha_1se`. Train a lasso model with this regularization parameter value and save this model as `lasso_fit_4b`. 

Lastly, extract the names of the selected variables (i.e. the variables corresponding to the non-zero coefficients of the lasso fit with chosen value for the regularization parameter $\lambda$) into an object `selected_names_4b` and save the total number of selected variables as `num_selected_4b`.

In [5]:
# CODE_CHUNK code_chunk_14
from sklearn.linear_model import LassoCV, Lasso
from sklearn.model_selection import KFold

# Implement cross-validation
cv = KFold(n_splits=10, shuffle=True, random_state=5)
lasso_cv_4b = LassoCV(cv=cv, random_state=5)
lasso_cv_4b.fit(X_4a, y_4a)

# Get tuning parameter obtained with one-S.E. rule.
mean_mse      = np.mean(lasso_cv_4b.mse_path_, axis=1)
stderr_mse    = np.std(lasso_cv_4b.mse_path_, axis=1)
idx_alpha_1se = np.where(mean_mse <= (min(mean_mse) + stderr_mse))[0][-1]
alpha_1se     = lasso_cv_4b.alphas_[idx_alpha_1se]

# Train Lasso model with optimal tuning parameter 
lasso_fit_4b = Lasso(alpha=alpha_1se)
lasso_fit_4b.fit(X_4a, y_4a)

# Extract non-zero coefficients
coef_nonzero      = np.where(lasso_fit_4b.coef_ != 0)[0]

selected_names_4b = np.array(X_varnames_4a)[coef_nonzero]
num_selected_4b   = len(coef_nonzero)

print("Selected Feature Names:", selected_names_4b)
print("Number of Selected Features:", num_selected_4b)

Selected Feature Names: ['PHONEother' 'KITCHEN2' 'MOBILTYPother' 'WINTEROVENother'
 'WINTERELSPother' 'NEWC1' 'DISH2' 'WASH2' 'DRY2' 'REFR2' 'BATHS' 'BEDRMS'
 'DENS' 'FAMRM' 'HALFB' 'KITCH' 'LIVING' 'RECRM' 'BUILT' 'LOT' 'UNITSF'
 'CLIMB' 'DIRAC' 'AIRSYS2' 'WELDUSother' 'STEAM1' 'STEAM2' 'FRPLother'
 'FPLWK2' 'FPINSother' 'DISPL2' 'TRASH2' 'TYPEother' 'ENOEAPP2' 'ECNTAIR1'
 'ECNTAIRother' 'EAIRCother' 'EHEATUT2' 'EHEATUTother' 'EFRIDGE2'
 'EWASHR2' 'EWASHRother' 'EDISHWR2' 'EDISHWRother' 'ETRSHCP1' 'ETRSHCP2'
 'AIR' 'NUMAIR' 'WATERother' 'WATERDother' 'HOTPIP2' 'SEWDISTPother'
 'SEWDUSother' 'KEXCLUother' 'INCPother' 'BUSPER' 'EXCLUS' 'LAUNDY'
 'OTHRUN' 'DRSHOP2' 'CONDO3' 'CELLAR4' 'CELLARother' 'WHNGET'
 'FRSTOCother' 'PREOCCother' 'EBARother' 'OTBUP1' 'OTBUP2' 'NUNITS'
 'PLUGSother' 'OWNLOTother' 'ROOMS' 'PLUMB2' 'ZADEQother' 'LEAKother'
 'WTRHRLother' 'RATSother' 'EGOODother' 'HOWH' 'BSINK2' 'TOILET2'
 'ELEVWK1' 'EROACH2' 'EVROD2' 'ROACHFRQother' 'CRACKS2' 'EBOARDother'
 'EBROKEothe

### Task 4c) 
Next, we do controlled variable selection using knock-offs. In python, this is implemented via the `knockpy` package can be accessed through PyPI.
The central fucntion for conducting knock-off variable selection is `KnockoffFilter()`, whose [documentation](https://amspector100.github.io/knockpy/apiref.html) provides you with an overview of its capabilities.

First, call `KnockoffFilter()` to specify the setup of our variable selection exercise. More specifically, we want to base  
The statistic that we want to use to do variable selection is
$$
Z_i  =max \{\lambda:\beta_j(\lambda) \neq 0\},
$$
as suggested in the seminal paper of Candes and Barber (2015). We also use the fixed-X option for knockoffs. Save the resulting specification object as `knockoff_filter_4c`.

Second, apply the `forward()`-method to `knockoff_filter_4c` to run the knockoff filter. I order to select the correct arguments, recall that we want to select predictors of log houes prices `y_4a` from the whose set of predictor candidates `X_4a`. Now, we want to control the probability of making a mistake with an FDR of 15%.

Lastly, save the names of the selected variables in an object `selected_names_4c` and the number of selected variables as `number_selected_4c`. How does the number selected variables in Task 4b compare to the number that you have now? Write you answer into the string variable `num_selected_compare_4c`.

*Note:* The *knockpy* package deos not deal particularly well with large datasets. As a result, you might get a memory error. In this (likely) case, use fewer data points of `X_4a` and `y_4a`. To a certain degree, you can also free up unused memory using the `collect()` function in the `gc` package.

In [20]:
# CODE_CHUNK code_chunk_15
import knockpy
#Even after clearing memory my desktop could only handle 85% of the dataset
np.random.seed(5)
n_rows = X_4a.shape[0]
subset_idx = np.random.choice(n_rows, size=int(0.85*n_rows), replace=False)
X_small = X_4a[subset_idx, :]
y_small = y_4a[subset_idx]
# 1.
knockoff_filter_4c = knockpy.KnockoffFilter(
    fstat="lasso", 
    ksampler="fx"
)

# 2.
knockoff_result_4c = knockoff_filter_4c.forward(X_small, y_small, fdr=0.15)
selected_names_4c = np.array(X_varnames_4a)[knockoff_result_4c.astype(int)==1]
number_selected_4c = selected_names_4c.size

print("Selected Feature Names:", selected_names_4c)
print("Number of Selected Features:", number_selected_4c)

# 3. 
num_selected_compare_4c = "Compared to 112 of 117 features selected in 4b, we now only have 69 features selected."

Selected Feature Names: ['MOBILTYPother' 'WINTEROVENother' 'NEWC1' 'DISH2' 'DRY2' 'BATHS' 'BEDRMS'
 'DENS' 'HALFB' 'KITCH' 'LIVING' 'BUILT' 'LOT' 'UNITSF' 'CLIMB' 'AIRSYS2'
 'WELDUSother' 'FPLWK2' 'FPINSother' 'DISPL2' 'TRASH2' 'TYPEother'
 'ENOEAPP2' 'EHEATUT2' 'EFRIDGE2' 'EWASHR2' 'EDISHWR2' 'EDISHWRother'
 'AIR' 'NUMAIR' 'WATERDother' 'SEWDUSother' 'INCPother' 'BUSPER' 'LAUNDY'
 'CONDO3' 'CELLAR4' 'CELLARother' 'WHNGET' 'FRSTOCother' 'PREOCCother'
 'EBARother' 'OTBUP1' 'OTBUP2' 'OWNLOTother' 'ROOMS' 'ZADEQother'
 'WTRHRLother' 'RATSother' 'EGOODother' 'HOWH' 'BSINK2' 'TOILET2'
 'ELEVWK1' 'EVROD2' 'ROACHFRQother' 'CRACKS2' 'HOLES2' 'FREEZEother'
 'IFSEWother' 'OTHCLDother' 'NOWIREother' 'REGION2' 'REGION3' 'REGION4'
 'METROother' 'UNITSFMISSother' 'HOWHMISSother' 'NUMTLTMISS0']
Number of Selected Features: 69
